In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import glob

hives_ids = ["smrpiclient7", "smrpiclient6", "smrpiclient3"]

hive_under_analysis = hives_ids[0]
timezone_offset_hours = 2

# Data Preprocessing

Here we load train data sound samples and prepare spectrogram, periodogram and mfcc features (along with some data to visualize this). We should provide data with **utc timestamps** as it will be shifted with `timezone_offset_hours` var. What we also do is remove those samples which has strange rms signal. Threshold 0.8 was chosen based on `plot_distribution` output.

In [2]:
import struct
import math
import numpy as np
import librosa
import librosa.display

from tqdm import tqdm
from datetime import datetime, timedelta
from scipy.io import wavfile
from scipy import signal
from scipy.fftpack import fft
from scipy.signal import decimate

import matplotlib.pyplot as plt
%matplotlib widget

sound_time_ms = 2000
# ~93 ms for fft window
nfft = 4096
# ~34% overlapping
hop_len = (nfft//3) + 30
# This can be manipulated to adjust number of bins for conv layer
fmax = 2750

hives_data = []
rmses = {}
max_to_norm = 0
for idx, hive_id in enumerate(hives_ids):
    sound_files = [f for f in glob.glob(f"..\\measurements\\smartulav2\\{hive_id}_*\\*.wav")]
    print(f"Sound data preparation for hive: {hive_id} which has {len(sound_files)} recordings...", end=' ', flush=True)
    rmses[hive_id] = []
    for file in tqdm(sound_files):
        sample_rate, sound_samples = wavfile.read(file)
        sound_samples = sound_samples.T[0]/(2.0**31)
        rms = np.sqrt(sum(sound_samples**2)/len(sound_samples))
        if(rms < 0.7):    # that threshold was observed from plot_distribution() function
            rmses[hive_id].append(rms)
            
            mfccs = librosa.feature.mfcc(y=sound_samples, sr=sample_rate, n_fft=nfft, hop_length=hop_len, n_mfcc=13)
            np_mfcc_avg = np.mean(mfccs, axis=1)

            spectrogram = librosa.core.stft(sound_samples, n_fft=nfft, hop_length=hop_len)
            spectrogram_magnitude = np.abs(spectrogram)
            spectrogram_phase = np.angle(spectrogram)
            spectrogram_db = librosa.amplitude_to_db(spectrogram_magnitude, ref=np.max)
            frequencies = librosa.fft_frequencies(sr=sample_rate, n_fft=nfft)
            times = (np.arange(0, spectrogram_magnitude.shape[1])*hop_len)/sample_rate

            freq_slice = np.where((frequencies < fmax))
            frequencies = frequencies[freq_slice]
            spectrogram_db = spectrogram_db[freq_slice, :][0]    
            spectrogram_mean = np.mean(spectrogram_db, axis=1)

#             spectrogram_db_decimated = decimate(spectrogram_db.T, 4).T
#             frequencies_decimated = decimate(frequencies, 4)
            
            filename = file.rsplit('\\', 1)[-1]
            utc_timestamp = filename[filename.index('-')+1:].rsplit(".wav")[0]
            sound_datetime = datetime.strptime(utc_timestamp, '%Y-%m-%dT%H-%M-%S') + timedelta(hours=timezone_offset_hours)
            hives_data.append([sound_datetime, hive_id, sound_samples, [frequencies, times, spectrogram_db], np_mfcc_avg, spectrogram_mean])
    print(" done.")

print(f"got full dataset of {len(hives_data)} sound samples")

Sound data preparation for hive: smrpiclient7 which has 3367 recordings... 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3367/3367 [01:52<00:00, 30.00it/s]

 done.
Sound data preparation for hive: smrpiclient6 which has 3172 recordings... 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3172/3172 [01:48<00:00, 29.22it/s]

 done.
Sound data preparation for hive: smrpiclient3 which has 602 recordings... 


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 602/602 [00:17<00:00, 33.70it/s]

 done.
got full dataset of 7033 sound samples


In [3]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sc = StandardScaler()
mm = MinMaxScaler()

# extract every spectrogram from hives data list
spectrograms = [hive_data[3][2] for hive_data in hives_data]
# standarize every periodogram from stft so data for spectrogram will have zero mean and unit variance
standarized_spectrograms = [sc.fit_transform(spectrogram.T).T for spectrogram in spectrograms]
# scale every standarized periodogram 
scaled_spectrogram = [mm.fit_transform(spectrogram_stan.T).T for spectrogram_stan in standarized_spectrograms]
# get datatime, names and mfcc
datetimes = [hive_data[0] for hive_data in hives_data]
names = [hive_data[1] for hive_data in hives_data]
mfccs = [hive_data[4] for hive_data in hives_data]
# standarize and scale periodogram for sounds 
periodograms_mean = [data[5] for data in hives_data]
standarized_periodograms = StandardScaler().fit_transform(periodograms_mean)
scaled_periodograms = MinMaxScaler().fit_transform(standarized_periodograms)

sounds = list(zip(scaled_spectrogram, mfccs, scaled_periodograms, datetimes, names))

sounds_data = pd.DataFrame(sounds, columns=['spectrogram', 'mfccs', 'periodogram', 'datetime', 'name'])
sounds_data['datetime'] = pd.to_datetime(sounds_data['datetime'])
sounds_hive_data = sounds_data[sounds_data['name'] == hive_under_analysis]

print(f"Got dataset of size: {len(sounds)}")

Got dataset of size: 7033


## Train BASIC AE

Here we train basic fully connected autoencoder on data from particular hive

In [ ]:
ae_basic_train, ae_basic_val = prepare_dataset1d(sounds_hive_data['periodogram'], train_ratio=0.8)

dl_aebasic_train = tdata.DataLoader(ae_basic_train, batch_size=32, shuffle=True)
dl_aebasic_val = tdata.DataLoader(ae_basic_val, batch_size=32, shuffle=True)

modelBasicAE = BasicAutoencoder()
modelBasicAE = train_model(modelBasicAE, learning_rate=1e-3, weight_decay=1e-5, num_epochs=100, patience=20, dataloader_train=dl_aebasic_train, dataloader_val=dl_aebasic_val)

## Train CONV AE

Here we train convolutional autoencoder on data from particular hive

In [4]:
import torch 
from torch.utils import data as tdata

train_data_size = (sounds_hive_data.shape[0]*90)//100
val_data_size = sounds_hive_data.shape[0] - train_data_size

dataset_tensor = torch.Tensor(sounds_hive_data['spectrogram'].values.tolist())
print(f"Dataset shape: {dataset_tensor.shape}")
print(f"Train set size: {train_data_size}")
print(f"Validation set size: {val_data_size}")

# add one extra dimension as it is required for conv layer
dataset_tensor = dataset_tensor[:, None, :, :] 
dataset = tdata.TensorDataset(dataset_tensor)
train_set, val_set = torch.utils.data.random_split(dataset, [train_data_size, val_data_size])

dataloader_train = tdata.DataLoader(train_set, batch_size=4, shuffle=True)
dataloader_val = tdata.DataLoader(val_set, batch_size=4, shuffle=True)

Dataset shape: torch.Size([3367, 256, 64])
Train set size: 3030
Validation set size: 337


In [ ]:
del modelConvAE

In [ ]:
%matplotlib widget

import sys
import matplotlib.pyplot as plt

num_epochs = 100
learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# del modelConvAE

modelConvAE = ConvAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(modelConvAE.parameters(), lr=learning_rate, weight_decay=1e-6)

# monitor training loss per batch
train_loss = []
# monitor validation loss per batch
val_loss = []
# save avg train losses for early stopping visualization
avg_train_loss = []
# save avg train losses for early stopping visualization
avg_val_loss = [] 
# patience when stop training
patience = 20
# counter for patience in early sotpping
patience_counter = 0
# best validation score
best_val_loss = -1
# model checkpoint filename
checkpoint_filename = 'checkpoint.pth'
# early stopping epoch
win_epoch = 0
    
for epoch in range(1, num_epochs+1):    
    ###################
    # train the model #
    ###################
    modelConvAE.train()
    for data in dataloader_train:
        # transfer data to device
        spectrogram = data[0].to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass
        outputs = modelConvAE(spectrogram)
        # calculate the loss
        loss = criterion(outputs, spectrogram)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss.append(float(loss.item()))
        
    ###################
    # val the model   #
    ###################
    modelConvAE.eval()
    for val_data in dataloader_val:
        # transfer data to device
        spectrogram_val = val_data[0].to(device)
        # forward pass
        voutputs = modelConvAE(spectrogram_val)
        # calculate the loss
        vloss = criterion(voutputs, spectrogram_val)
        # update running val loss
        val_loss.append(float(vloss.item()))
    
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = np.average(train_loss)
    val_loss = np.average(val_loss)
    avg_train_loss.append(train_loss)
    avg_val_loss.append(val_loss)
    
    epoch_len = len(str(num_epochs))
    # print avg training statistics 
    print(f'[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] train_loss: {train_loss:.5f} valid_loss: {val_loss:.5f}', end=' ', flush=True)
    
    if val_loss < best_val_loss or best_val_loss == -1:
        # new checkpoint
        print("checkpoint!")
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(modelConvAE.state_dict(), checkpoint_filename)
        win_epoch = epoch
    elif patience_counter >= patience:
        print("early stopping.")
        print(f"=> loading checkpoint {checkpoint_filename}")
        device = torch.device("cuda")
        modelConvAE.load_state_dict(torch.load(checkpoint_filename))
        break
    else:
        print(".")
        patience_counter = patience_counter + 1
        
    # clear batch losses
    train_loss = []
    val_loss = []

fig = plt.figure(figsize=(10,5))
plt.plot(np.arange(1, epoch + 1), avg_train_loss, 'r', label="train loss")
plt.plot(np.arange(1, epoch + 1), avg_val_loss, 'b', label="validation loss")
plt.axvline(win_epoch, linestyle='--', color='g',label='Early Stopping Checkpoint')
plt.legend(loc='best')
plt.grid(True)
plt.show()

[  1/100] train_loss: 0.06261 valid_loss: 0.04426 checkpoint!


# Encode data

In [ ]:
encoded_data = conv2d_encode(modelConvAE, scaled_spectrogram)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import random

idx = random.randint(0, len(hives_data) - 1)
with torch.no_grad():
    fig, axs = plt.subplots(2, 1)
    frequencies = librosa.fft_frequencies(sr=sample_rate, n_fft=nfft)
    freq_slice = np.where((frequencies < fmax))
    frequencies = frequencies[freq_slice]
    times = (np.arange(0, spectrogram_magnitude.shape[1])*hop_len)/sample_rate    
    elem = scaled_spectrogram[idx]
    elem = elem[None, None,: ,:]
    elem = torch.Tensor(elem)

    axs[0].pcolormesh(times, frequencies, scaled_spectrogram[idx])
    axs[1].pcolormesh(times, frequencies, modelConvAE(elem.to(device)).cpu().numpy().squeeze())

# Add temperature/humidity/gas

In [ ]:
start_time = '2020-08-10 00:00:00'
end_time = '2020-09-16 00:00:00'
print(f"extracting data for hive under analysis: {hive_under_analysis} from {start_time} to {end_time}...")

df_hives_sound = pd.DataFrame(sounds_data)
df_hive_sound_ua = df_hives_sound[(df_hives_sound['name'] == hive_under_analysis)
                                 & (df_hives_sound['datetime'] > start_time)
                                 & (df_hives_sound['datetime'] < end_time)]
df_hive_sound_ua.set_index('datetime', inplace=True)
print(f"-> prepared base of {df_hive_sound_ua.count()['spectrogram']} numer of sound spectrum <-")

df_hive_temperature_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-temperature.csv', hive_under_analysis, start_time, end_time, 'temperature')
df_hive_humidity_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-humidity.csv', hive_under_analysis, start_time, end_time, 'humidity')
df_hive_alcohol_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-alcohol.csv', hive_under_analysis, start_time, end_time, 'alcohol')
df_hive_aceton_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-aceton.csv', hive_under_analysis, start_time, end_time, 'aceton')
df_hive_amon_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-jon-amonowy.csv', hive_under_analysis, start_time, end_time, 'jon-amonowy')
df_hive_toluen_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-toluen.csv', hive_under_analysis, start_time, end_time, 'toluen')
df_hive_co2_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-co2.csv', hive_under_analysis, start_time, end_time, 'co2')
df_hive_siarkowodor_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-siarkowodor.csv', hive_under_analysis, start_time, end_time, 'siarkowodor')
df_hive_metanotiol_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-metanotiol.csv', hive_under_analysis, start_time, end_time, 'metanotiol')
df_hive_trimetyloamina_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-trimetyloamina.csv', hive_under_analysis, start_time, end_time, 'trimetyloamina')
df_hive_wodor_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-wodor.csv', hive_under_analysis, start_time, end_time, 'wodor')
df_hive_co_ua = read_sensor_data('..//measurements//smartulav2//sulmin-10082020-15092020-inside-co.csv', hive_under_analysis, start_time, end_time, 'co')


Check autocorrelation for specific features

In [ ]:
# change sound base (mfccs dataframe or conv ae dataframe)
sound_base_feature = 'conv_ae' 

df_hive_data = merge_dataframes_ontimestamp(df_hive_sound_ua,
                                            df_hive_temperature_ua, df_hive_humidity_ua,
                                            df_hive_alcohol_ua, df_hive_aceton_ua, df_hive_amon_ua, df_hive_toluen_ua, df_hive_co2_ua,
                                            df_hive_siarkowodor_ua, df_hive_metanotiol_ua, df_hive_trimetyloamina_ua, df_hive_wodor_ua,
                                            df_hive_co_ua)

if sound_base_feature == 'conv_ae':
    print(f"encoding sound samples with autoencoder...", end=' ')
    df_hive_data['conv_ae'] = conv2d_encode(modelConvAE, df_hive_data['spectrogram'].to_list())
    print(f"finish!")

df_hive_data['feature_vector'] = merge_columns(df_hive_data, [sound_base_feature, 'humidity', 'temperature',
                                                'alcohol', 'aceton', 'jon-amonowy', 'toluen', 'co2',
                                                'trimetyloamina',
#                                                 'siarkowodor', 'metanotiol', 'trimetyloamina', 'wodor',
                                                'co'])

# SVM classification 

In [ ]:
from sklearn.preprocessing import StandardScaler

start_hours = [20, 21, 22, 23, 0, 1, 2, 3, 4]

# data for convolutional autoencoder
pd_convae_data = pd.DataFrame(df_hive_data)
pd_convae_data['feature_vector'] = StandardScaler().fit_transform(df_hive_data['feature_vector'].values.tolist()).tolist()

# data for mfcc 
mfccs = [hive_data[4] for hive_data in hives_data if hive_data[1] == hive_under_analysis]
mfccs = StandardScaler().fit_transform(mfccs)
datetimes = [hive_data[0] for hive_data in hives_data if hive_data[1] == hive_under_analysis]
mfccs_data = list(zip(datetimes, mfccs))
pd_mfcc_data = pd.DataFrame(mfccs_data, columns=['datetime', 'mfcc'])
pd_mfcc_data.set_index('datetime', inplace=True)

print('calculating mfccs match...', end=' ', flush=True)
mfcc_accs = search_best_night_day(pd_mfcc_data, 'mfcc', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(mfcc_accs)}/{len(mfcc_accs[0])}')
print('calculating feature vector match...', end=' ', flush=True)
ae_accs = search_best_night_day(pd_convae_data, 'feature_vector', days_as_test=10, start_hours=start_hours, max_shift=6, verbose=0)
print(f'done. {len(ae_accs)}/{len(ae_accs[0])}')

plot_hour_shift(mfcc_accs, ae_accs, xticklabels=[str(start_hour) for start_hour in start_hours])

Visualize on 2D map, we basically perform TSNE and PCA dimension reduction in order to visualize night and day. Probably this will be not efficent but it is worth to hive a shot.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

start_hour = 23
end_hour = 2

reduce_df = pd.DataFrame(df_hive_data)
reduce_df['feature_vector'] = StandardScaler().fit_transform(df_hive_data['feature_vector'].values.tolist()).tolist()

reduced_ae_pca = PCA(n_components=2).fit_transform(reduce_df['feature_vector'].values.tolist())
reduced_ae_tsne =  TSNE(n_components=2, perplexity=100, learning_rate=500).fit_transform(reduce_df['feature_vector'].values.tolist())
is_night_list = (reduce_df.index.hour >= start_hour) | (reduce_df.index.hour <= end_hour)
                
colors = ['red', 'green', 'blue', 'yellow']
labels = ['day', 'night']

fig, axs = plt.subplots(2, figsize=(10,10))

axs[0].scatter(x=[data[0] for data in reduced_ae_pca],
               y=[data[1] for data in reduced_ae_pca],
               c=[colors[night] for night in is_night_list],
              alpha=0.3)
axs[0].set_title('PCA')

axs[1].scatter(x=[data[0] for data in reduced_ae_tsne],
               y=[data[1] for data in reduced_ae_tsne],
               c=[colors[night] for night in is_night_list],
              alpha=0.3)
axs[1].set_title('TSNE')

plt.show()

In [ ]:
pca = PCA().fit(reduce_df['feature_vector'].values.tolist())

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

# ----- Functions/Classes -----

Function for plotting sepctrogram by function.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib widget

def plot_spectrogram(frequency, time_x, spectrocgram, title):
    fig = plt.figure(figsize=(6,4))
    plt.title(title)
    plt.pcolormesh(time_x, frequency, spectrocgram)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

Class for convolutional autoencoder and some helper functions.

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class View(nn.Module):
    """ Function for nn.Sequentional to reshape data """
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

def conv2d_block(in_f, out_f, *args, **kwargs):
    """ Function for building convolutional block

        Attributes
            in_f - number of input features
            out_f - number of output features
    """
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU(),
        nn.Dropout2d(p=0.2)
    )

def conv2d_transpose_block(in_f, out_f, *args, **kwargs):
    """ Function for building transpose convolutional block
        
        Attributes
            in_f - number of input features
            out_f - number of output features
    """
    return nn.Sequential(
        nn.ConvTranspose2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU(),
        nn.Dropout2d(p=0.2)
    )

######################################
#                                    #
#   Main convolutional autoencoder   #
#                                    #
######################################
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        
        ## encoder layers ##
        self.encoder = nn.Sequential(
            # [1x256x64] => [64x256x64]
            conv2d_block(1, 128, kernel_size=3, padding=1),
            # [64x256x64] => [64x128x32]
            nn.MaxPool2d(2, 2),
            # [64x128x32] => [32x128x32]
            conv2d_block(128, 64, kernel_size=3, padding=1),
            # [32x128x32] => [32x64x16]
            nn.MaxPool2d(2, 2),
            # [32x64x16] => [16x64x16]
            conv2d_block(64, 32, kernel_size=3, padding=1),
            # [16x64x16] => [16x32x8]
            nn.MaxPool2d(2, 2),
            # [16x32x8] => [4x32x8]
            conv2d_block(32, 16, kernel_size=3, padding=1),
            # [4x32x8] => [4x16x4]
            nn.MaxPool2d(2, 2),
            # [4x16x4] => [1x256]
            nn.Flatten(),
            # [1x256] => [1x64]
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        ## decoder layers ##
        self.decoder = nn.Sequential(
            # [1x64] => [1x256]
            nn.Linear(128, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            # [1x256] => [4x16x4]
            View([-1, 16, 16, 4]),
            # [4x16x4] => [16x32x8]
            conv2d_transpose_block(16, 32, kernel_size=2, stride=2),
            # [16x32x8] => [32x64x16]
            conv2d_transpose_block(32, 64, kernel_size=2, stride=2),
            # [32x64x16] => [64x128x32]
            conv2d_transpose_block(64, 128, kernel_size=2, stride=2),
            # [64x128x32] => [1x256x64]
            nn.ConvTranspose2d(128, 1, kernel_size=2, stride=2),
            nn.Sigmoid()
        )


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        
        return x

basic fully connected autoencoder

In [7]:
import torch
from torch import nn

class BasicAutoencoder(nn.Module):
    def __init__(self):
        super(BasicAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features=256, out_features=128),
            nn.BatchNorm1d(num_features=128),
            nn.ReLU(True),
            nn.Linear(in_features=128, out_features=64),
            nn.BatchNorm1d(num_features=64),
            nn.ReLU(True),
            nn.Linear(in_features=64, out_features=32),
            nn.BatchNorm1d(num_features=32),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(in_features=32, out_features=64),
            nn.BatchNorm1d(num_features=64),
            nn.ReLU(True),
            nn.Linear(in_features=64, out_features=128),
            nn.BatchNorm1d(num_features=128),
            nn.ReLU(True),
            nn.Linear(in_features=128, out_features=256),
            nn.BatchNorm1d(num_features=256),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

Function for extracting ecoded data from trained model.

In [8]:
def conv2d_encode(model, data_intput):
    """ Function for encoding data and returning encoded """
    dataset_tensor = torch.Tensor(data_intput)
    dataset_tensor = dataset_tensor[:, None, :, :]
    dataset_tensor = tdata.TensorDataset(dataset_tensor)
    dataset = tdata.DataLoader(dataset_tensor, batch_size=32, shuffle=True)
    encoded_data = []
    
    model.eval()
    with torch.no_grad():
        for data in dataset:
            periodograms = data[0].to(device)
            output = modelConvAE.encoder(periodograms).cpu().numpy().squeeze()
            encoded_data.extend(output)
    
    return encoded_data

Function for preparing dataset.

In [9]:
from torch import Tensor
from torch.utils import data as tdata

def prepare_dataset1d(data_df, train_ratio):
    """ Function for preparing dataset for autoencoder 
    
        attributes: data_df - pandas dataframe column
        attributes: train_ratio - radio of train set size
        return train_dataset, test_dataset
    """
    train_data_size = int(data_df.shape[0]*train_ratio)
    val_data_size = data_df.shape[0] - train_data_size

    dataset_tensor = torch.Tensor(data_df.values.tolist())
    print(f"Dataset shape: {dataset_tensor.shape}")
    print(f"Train set size: {train_data_size}")
    print(f"Validation set size: {val_data_size}")

    # add one extra dimension as it is required for conv layer
    # dataset_tensor = dataset_tensor[:, None, :] 
    dataset = tdata.TensorDataset(dataset_tensor)
    train_set, val_set = tdata.random_split(dataset, [train_data_size, val_data_size])
    
    return train_set, val_set

Function for model learning with early stopping and plotting tran graph

In [ ]:
def train_model(model, learning_rate, weight_decay, num_epochs, patience,
                dataloader_train, dataloader_val, checkpoint_name='checkpoint.pth'):
    """ 
    Function for training model 
    
    attribute: model - model which should be trained
    attribute: learning_rate - learning rate for the model
    attribute: weight_decay - weight decay for learning
    attribute: num_epochs - number epochs 
    attribute: patience - patience for early stopping
    attribute: dataloader_train - train data loader
    attribute: dataloader_val - validation data loader
    attribute: checkpoint_name - checkpoint name for early stopping
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # monitor training loss per batch
    train_loss = []
    # monitor validation loss per batch
    val_loss = []
    # save avg train losses for early stopping visualization
    avg_train_loss = []
    # save avg train losses for early stopping visualization
    avg_val_loss = [] 
    # counter for patience in early sotpping
    patience_counter = 0
    # best validation score
    best_val_loss = -1
    # model checkpoint filename
    checkpoint_filename = checkpoint_name
    # early stopping epoch
    win_epoch = 0
    
    # pass model to gpu if is available
    model.to(device)
    
    for epoch in range(1, num_epochs+1):    
        ###################
        # train the model #
        ###################
        model.train()
        for data in dataloader_train:
            # transfer data to device
            input_data = data[0].to(device)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass
            outputs = model(input_data)
            # calculate the loss
            loss = criterion(outputs, input_data)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss.append(loss.item())

        ###################
        # val the model   #
        ###################
        model.eval()
        for val_data in dataloader_val:
            # transfer data to device
            input_data_val = val_data[0].to(device)
            # forward pass
            val_outputs = model(input_data_val)
            # calculate the loss
            vloss = criterion(val_outputs, input_data_val)
            # update running val loss
            val_loss.append(vloss.item())

        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_loss)
        val_loss = np.average(val_loss)
        avg_train_loss.append(train_loss)
        avg_val_loss.append(val_loss)

        epoch_len = len(str(num_epochs))
        # print avg training statistics 
        print(f'[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] train_loss: {train_loss:.5f} valid_loss: {val_loss:.5f}', end=' ', flush=True)

        if val_loss < best_val_loss or best_val_loss == -1:
            # new checkpoint
            print("checkpoint!")
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), checkpoint_filename)
            win_epoch = epoch
        elif patience_counter >= patience:
            print("early stopping.")
            print(f"=> loading checkpoint {checkpoint_filename}")
            device = torch.device("cuda")
            modelConvAE.load_state_dict(torch.load(checkpoint_filename))
            break
        else:
            print(".")
            patience_counter = patience_counter + 1

        # clear batch losses
        train_loss = []
        val_loss = []

    fig = plt.figure(figsize=(10,5))
    plt.plot(np.arange(1, epoch + 1), avg_train_loss, 'r', label="train loss")
    plt.plot(np.arange(1, epoch + 1), avg_val_loss, 'b', label="validation loss")
    plt.axvline(win_epoch, linestyle='--', color='g',label='Early Stopping Checkpoint')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()
    
    return model

Function for reading sensor data from file and some helper function for merging data.

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def read_sensor_data(filename, hive_sn, start_time, end_time, sensor_column_name):
    """ Function for reading smartula sensor file (from grafana) and build pandas dataframe """
    df_sensor_data = pd.read_csv(filename, skiprows=1, sep=";")
    
    if hive_sn not in hives_ids:
        print(f"Hive {hive_sn} is not in hives_ids set! Returning empty dataframe")
        return pd.DataFrame()
    
    # change series column to be coherent with sounds
    for hive in hives_ids:
        df_sensor_data.loc[df_sensor_data['Series'].str.contains(hive[2:]), 'Series'] = hive

    # change column names to match sound
    df_sensor_data.columns = ['name', 'datetime', sensor_column_name]
    # convert timestamp to pandas timestamp
    df_sensor_data['datetime'] = [(datetime.strptime(date_pd[:-6], '%Y-%m-%dT%H:%M:%S') +
                                   timedelta(hours=timezone_offset_hours)) for date_pd in df_sensor_data['datetime'].values.tolist()]
    
    df_sensor_data = df_sensor_data[(df_sensor_data['name'] == hive_sn) & (df_sensor_data['datetime'] > start_time) & (df_sensor_data['datetime'] < end_time)]
    df_sensor_data.set_index('datetime', inplace=True)
    print(f"got {df_sensor_data[sensor_column_name].count()} of {sensor_column_name} samples")
    
    return df_sensor_data

def merge_dataframes_ontimestamp(df_merge_to, *args):
    """ Merging dataframes to df_merge_to """
    df_hive_data_ua = df_merge_to
    for dataframe in args:
        df_hive_data_ua = pd.merge(df_hive_data_ua, dataframe.reindex(df_hive_data_ua.index, method='nearest'), on=['datetime', 'name'])
        
    return df_hive_data_ua

flatten util

In [ ]:
import collections

def flatten(x):
    if isinstance(x, collections.abc.Iterable):
        return [a for i in x for a in flatten(i)]
    else:
        return [x]
    
def merge_columns(dataframe, column_names):
    """ Function for merging columns with irregular size """
    return [flatten(val) for val in  dataframe[column_names].values.tolist()]
    

Function for performing grid search on best OneClasSVM on day/night classification and visualizing results

In [ ]:
from sklearn.svm import SVC
import pandas as pd

  
def plot_hour_shift(*args, xticklabels):
    """ Function for plotting n-hour shift """
    fig, axs  = plt.subplots(len(args[0])//2, 2, figsize=(10,8))
    fig.subplots_adjust(hspace=0.7)
    
    colors = ['ro', 'bx', 'go', 'yx', 'ko']
    
    if len(args) > len(colors):
        print('warning your accuracies are bigger than colors for plot!')
    
    for feature_idx, accuracy in enumerate(args):
        for acc_idx, acc_in_shift in enumerate(accuracy):
            axs[acc_idx//2][acc_idx%2].plot(acc_in_shift, colors[feature_idx])
            axs[acc_idx//2][acc_idx%2].grid()
            axs[acc_idx//2][acc_idx%2].set_xticks(np.arange(0, len(xticklabels), 1))
            axs[acc_idx//2][acc_idx%2].tick_params(axis='x', rotation=270)
            axs[acc_idx//2][acc_idx%2].set_xticklabels(xticklabels)
            axs[acc_idx//2][acc_idx%2].set_title(f'{acc_idx+1} hour long bee-night')
            axs[acc_idx//2][acc_idx%2].set_ylabel('SVM accuracy')
            axs[acc_idx//2][acc_idx%2].set_xlabel('Hour')
    fig.show()

def search_best_night_day(input_data, feature_name, days_as_test, start_hours, max_shift, verbose=0):
    """ Function performing One-class SVM
    
        attribute: train_data - pandas series dataframe
        attribute: feature_name - name of column from dataframe which will be used as feature
        attribute: days_test - number of last days which will be used to create train data
        attribute: start_hours - list with start hours
        attribute: max_shift - max shift in hours
    """
    max_accuracy = 0
    
    accs_per_shift = []
    final_accs = []

    for shift in range(1, max_shift+1):
        for start_hour in start_hours:
            data_to_svm = pd.DataFrame(input_data)
            data_to_svm.sort_index(inplace=True)
            
            end_hour = (start_hour + shift) % 24
            if end_hour > 12 or start_hour < max_shift:
                data_to_svm['is_night'] = (data_to_svm.index.hour >= start_hour) & (data_to_svm.index.hour <= end_hour)
            else:
                data_to_svm['is_night'] = (data_to_svm.index.hour >= start_hour) | (data_to_svm.index.hour <= end_hour)
                
            samples_in_day = data_to_svm[data_to_svm.index < (data_to_svm.index[0] + timedelta(days=1))].count()
            data_test = data_to_svm.tail(samples_in_day[0]*days_as_test)
            data_train = data_to_svm[~data_to_svm.isin(data_test)].dropna(how='all')
            
            train_data = data_train[feature_name].values.tolist()
            train_labels = data_train['is_night'].values.tolist()
            test_data = data_test[feature_name].values.tolist()
            test_labels = data_test['is_night'].values.tolist()
            
            if verbose > 0:
                print(f'learning with train data size: {len(train_data)} and test data size: {len(test_data)}')
                print(f'number of nights in train/test data: {sum(train_labels)}/{sum(test_labels)}')
            svc = SVC(kernel='rbf', class_weight='balanced', gamma='auto')
            svc.fit(train_data, train_labels)
            predicted = svc.predict(test_data)
            
            sum_correct = 0
            for idx, label_predicted in enumerate(predicted):
                if(label_predicted == int(test_labels[idx])):
                    sum_correct += 1

            accuracy = (sum_correct/len(test_labels)*100)
            if accuracy > max_accuracy:
                if verbose > 0:
                    print(f'new max acuuracy for {start_hour} to {end_hour}, accuracy: {accuracy:.2f}')
                max_accuracy = accuracy
            
            if verbose > 0:
                print(f'for night start at {start_hour} and end at {end_hour} got accuracy: {accuracy:.2f}')
                print('==============================================================================')
            
            accs_per_shift.append(accuracy)
        final_accs.append(accs_per_shift)
        accs_per_shift = []
        
    return final_accs

Function for calculating distribution for specific feature.

In [ ]:
import matplotlib

def plot_distribution(distribution_dict, bin_size):
    """ Plotting distribiution for dictionary elements"""
    colors = ['blue', 'green', 'red', 'yellow', 'black', 'pink', 'purple']
    rms_max = 0
    rms_min = 65535
    for k, v in rmses.items():
        if np.max(v) > rms_max:
            rms_max = np.max(v)
        if np.min(v) < rms_min:
            rms_min = np.min(v)
        
    plt.figure()
    for idx, (k, v) in enumerate(distribution_dict.items()):
        plt.hist(v, color=colors[idx%len(colors)], bins=int(np.abs(rms_max-rms_min)/bin_size))
    plt.show()

Part of code for calculating autocorrelaction for specific feature

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf

%matplotlib widget

features = ['conv_ae', 'humidity', 'temperature',
            'alcohol', 'aceton', 'jon-amonowy',
            'toluen', 'co2', 'siarkowodor',
            'metanotiol', 'trimetyloamina', 'wodor', 'co']

feature = features[12]
data_to_autocorr = df_hive_co_ua

roll_len = 3
interval = (data_to_autocorr.index[2] - data_to_autocorr.index[1]).seconds//60%60

y2 = data_to_autocorr[feature].rolling(window=roll_len).mean().values
y_corr = y2[roll_len:]
x_corelation = np.arange(start=0, step=2, stop=150)

fig, axes = plt.subplots(1, figsize=(8,5))
x = plot_acf(y_corr, lags=x_corelation, zero=False, ax=axes)
x_raw = acf(y_corr, nlags=150)
axes.set_title(f'{feature} autocorrelaction')
axes.set_xlabel(f'Lag (1 lag = {interval} minutes)') 
axes.set_ylabel('Correlation')
axes.set_xticks(np.arange(0, 151, step=10))

print(f'{feature} with max {max(x_raw[60:]):.2f} at {60 + np.argmax(x_raw[60:])}')

# temperature with max 0.74 at 93 (15 mint)
# humidity with max 0.58 at 92 (15 min)
# alcohol with max 0.53 at 134 (10 min)
# aceton with max 0.52 at 133 (10 min)
# jon-amonowy with max 0.57 at 133 (10 min)
# toluen with max 0.52 at 134 (10 min)
# co2 with max 0.54 at 133 (10 min)
# siarkowodor with max 0.16 at 142 (10 min)
# metanotiol with max 0.34 at 140 (10 min)
# trimetyloamina with max 0.56 at 138 (10 min)
# wodor with max 0.14 at 142 (10 min)
# co with max 0.62 at 134 (10 min)